In [52]:
import pandas as pd
from pandas.core.frame import DataFrame as pdf
import seaborn as sns
import matplotlib.pyplot as plt
import local_signals
import model
import dataset
from Bio import SeqIO
from sklearn.ensemble import RandomForestClassifier
import numpy as np
from scipy import stats
from features import Features
from Bio.Data import IUPACData

In [20]:
def read_and_validate_all_challenge_fasta_seq_and_id():
    # sequence, id = '', ''
    id_seq = {}
    sequence_count = len(list(SeqIO.parse(f'../datasets/fasta_files/challenge.fasta', 'fasta')))
    # print(f'Expecting 20 sequences in challenge.fasta. Found {sequence_count}')

    for record in SeqIO.parse(f'../datasets/fasta_files/challenge.fasta', 'fasta'):
        if set(record.seq) <= set(IUPACData.protein_letters):
            sequence = record.seq
            id = record.id
            # print(f'ID {id}, \n has sequence {sequence}')
            id_seq[id] = sequence
    return id_seq

id_seq = read_and_validate_all_challenge_fasta_seq_and_id()

In [34]:
def build_31_features_for_challenge(sequence):
    features_ary = np.zeros((1, 31), dtype=np.float32)

    feats = Features(sequence)
    glob_aa_comp = feats.global_aa_comp()
    features_ary[0, :20] = glob_aa_comp

    Nterm_50_gravy, Cterm_50_gravy = feats.local_hydrophobicity(sequence)
    features_ary[0, 20] = Nterm_50_gravy
    features_ary[0, 21] = Cterm_50_gravy

    features_ary[0, 22] = feats.mol_weight()
    features_ary[0, 23] = feats.mean_flexibility()
    features_ary[0, 24] = feats.aromaticity()
    features_ary[0, 25] = feats.isoelectric_point()
    features_ary[0, 26], features_ary[0, 27] = feats.net_charge_and_mnc()
    features_ary[0, 28], _ = feats.total_charge_and_mtc()
    features_ary[0, 29] = feats.mean_hydrophobicity()
    features_ary[0, 30] = len(sequence)

    return features_ary

all_20 = []
row_ids = []
chall_feats = np.zeros((20, 31))

for i, (id, seq) in enumerate(id_seq.items()):
    row_ids.append(id)
    chall_feats[i, :] = build_31_features_for_challenge(seq)

In [35]:
chall_feats.shape

(20, 31)

In [36]:
label_and_features = np.genfromtxt('../datasets/fiveClass.csv', delimiter=',')
# 76 cols. 75 features  + label col at 0, and 3 SP columns at end.
label_and_features.shape

(9401, 76)

In [37]:
# ONLY RUN ONCE!
# drop mtc, signal peptides and Nterm50 and Cterm50 aa comps.
cols_to_remove  = list(range(21, 61)) + [70, 73, 74, 75]
print(len(cols_to_remove))
label_and_features_less = np.delete(label_and_features, cols_to_remove, axis=1)
label_and_features_less.shape

44


(9401, 32)

In [39]:
X_train_31, X_test_31, y_train_31, y_test_31 = model.split_train_test(label_and_features_less)

In [40]:
# stick it back together
X_31 = np.concatenate([X_train_31, X_test_31], axis=0)
y_31 = np.concatenate([y_train_31, y_test_31], axis=0)
print(X_31.shape)
print(y_31.shape)

(9401, 31)
(9401,)


In [41]:
# train clf on all 9401 proteins dataset
clf, _ = model.train_RFC(X_31, y_31)

Trained in 7 secs


In [45]:
_20_preds = np.zeros((5, 20))
_20_preds.shape

_20_probs = np.zeros((5, 20))
_20_probs.shape

(5, 20)

In [50]:
probs_0 = clf.predict_proba(chall_feats)
_20_preds[0, :] = clf.predict(chall_feats)

probs_1 = clf.predict_proba(chall_feats)
_20_preds[1, :] = clf.predict(chall_feats)

probs_2 = clf.predict_proba(chall_feats)
_20_preds[2, :] = clf.predict(chall_feats)

probs_3 = clf.predict_proba(chall_feats)
_20_preds[3, :] = clf.predict(chall_feats)

probs_4 = clf.predict_proba(chall_feats)
_20_preds[4, :] = clf.predict(chall_feats)


In [57]:
arr_stack = np.stack((probs_0, probs_1, probs_2, probs_3, probs_4))
elementwise_mean = np.mean(arr_stack, axis=0)
elementwise_mean

array([[0.14, 0.27, 0.45, 0.11, 0.03],
       [0.06, 0.04, 0.05, 0.02, 0.83],
       [0.31, 0.05, 0.21, 0.39, 0.04],
       [0.38, 0.  , 0.  , 0.62, 0.  ],
       [0.65, 0.  , 0.04, 0.28, 0.03],
       [0.38, 0.08, 0.1 , 0.37, 0.07],
       [0.07, 0.44, 0.12, 0.14, 0.23],
       [0.15, 0.17, 0.3 , 0.16, 0.22],
       [0.27, 0.31, 0.15, 0.17, 0.1 ],
       [0.14, 0.2 , 0.22, 0.17, 0.27],
       [0.37, 0.08, 0.14, 0.31, 0.1 ],
       [0.33, 0.12, 0.11, 0.4 , 0.04],
       [0.18, 0.03, 0.13, 0.53, 0.13],
       [0.42, 0.01, 0.  , 0.56, 0.01],
       [0.16, 0.04, 0.05, 0.04, 0.71],
       [0.23, 0.01, 0.05, 0.69, 0.02],
       [0.23, 0.03, 0.06, 0.09, 0.59],
       [0.26, 0.02, 0.05, 0.64, 0.03],
       [0.25, 0.03, 0.23, 0.23, 0.26],
       [0.17, 0.15, 0.03, 0.05, 0.6 ]])

In [58]:
elementwise_mean *= 100
elementwise_mean

array([[14., 27., 45., 11.,  3.],
       [ 6.,  4.,  5.,  2., 83.],
       [31.,  5., 21., 39.,  4.],
       [38.,  0.,  0., 62.,  0.],
       [65.,  0.,  4., 28.,  3.],
       [38.,  8., 10., 37.,  7.],
       [ 7., 44., 12., 14., 23.],
       [15., 17., 30., 16., 22.],
       [27., 31., 15., 17., 10.],
       [14., 20., 22., 17., 27.],
       [37.,  8., 14., 31., 10.],
       [33., 12., 11., 40.,  4.],
       [18.,  3., 13., 53., 13.],
       [42.,  1.,  0., 56.,  1.],
       [16.,  4.,  5.,  4., 71.],
       [23.,  1.,  5., 69.,  2.],
       [23.,  3.,  6.,  9., 59.],
       [26.,  2.,  5., 64.,  3.],
       [25.,  3., 23., 23., 26.],
       [17., 15.,  3.,  5., 60.]])

In [60]:
_20_preds

array([[2., 4., 3., 3., 0., 0., 1., 2., 1., 4., 0., 3., 3., 3., 4., 3.,
        4., 3., 4., 4.],
       [2., 4., 3., 3., 0., 0., 1., 2., 1., 4., 0., 3., 3., 3., 4., 3.,
        4., 3., 4., 4.],
       [2., 4., 3., 3., 0., 0., 1., 2., 1., 4., 0., 3., 3., 3., 4., 3.,
        4., 3., 4., 4.],
       [2., 4., 3., 3., 0., 0., 1., 2., 1., 4., 0., 3., 3., 3., 4., 3.,
        4., 3., 4., 4.],
       [2., 4., 3., 3., 0., 0., 1., 2., 1., 4., 0., 3., 3., 3., 4., 3.,
        4., 3., 4., 4.]])

In [ ]:
_20_probs

In [22]:
chall_feats

array([[ 7.800e+01,  2.900e+01,  1.900e+01,  6.800e+01,  5.800e+01,
         1.070e+02,  2.900e+01,  8.700e+01,  5.800e+01,  7.800e+01,
         3.900e+01,  3.900e+01,  1.900e+01,  4.900e+01,  7.800e+01,
         9.700e+01,  3.900e+01,  2.900e+01,  0.000e+00,  0.000e+00,
        -7.000e+00, -1.900e+01,  1.100e+01,  9.990e+02,  5.800e+01,
         9.570e+02,  4.700e+02,  4.560e+02,  2.300e+01, -2.000e+01,
         1.030e+02],
       [ 9.700e+01,  1.500e+01,  2.400e+01,  5.800e+01,  7.300e+01,
         7.000e+01,  0.000e+00,  9.700e+01,  4.200e+01,  1.240e+02,
         4.800e+01,  3.000e+01,  2.100e+01,  2.700e+01,  2.400e+01,
         7.000e+01,  3.000e+01,  9.100e+01,  1.500e+01,  4.200e+01,
         8.400e+01,  1.340e+02,  3.700e+01,  9.830e+02,  1.300e+02,
         5.000e+02, -5.880e+02, -1.780e+02,  4.900e+01,  8.300e+01,
         3.300e+02],
       [ 7.400e+01,  4.000e+00,  1.100e+01,  5.900e+01,  1.500e+01,
         5.200e+01,  4.100e+01,  5.500e+01,  8.100e+01,  9.600e+01,
      

In [28]:
cols = ['A%', 'C%', 'D%', 'E%', 'F%', 'G%', 'H%', 'I%', 'K%', 'L%', 'M%', 'N%', 'P%', 'Q%', 'R%',
        'S%', 'T%', 'V%', 'W%', 'Y%', 'Nt50_hb', 'Ct50_hb', 'mw', 'flx', 'ar', 'ip', 'nc', 'mnc',
        'tc', 'hb', 'protein length']
print(len(cols))

31


In [29]:
cf_pdf = pd.DataFrame(chall_feats, columns=cols)
cf_pdf.index = row_ids
cf_pdf

,A%,C%,D%,E%,F%,G%,H%,I%,K%,L%,...,Ct50_hb,mw,flx,ar,ip,nc,mnc,tc,hb,protein length
SEQ01,78.0,29.0,19.0,68.0,58.0,107.0,29.0,87.0,58.0,78.0,...,-19.0,11.0,999.0,58.0,957.0,470.0,456.0,23.0,-20.0,103.0
SEQ02,97.0,15.0,24.0,58.0,73.0,70.0,0.0,97.0,42.0,124.0,...,134.0,37.0,983.0,130.0,500.0,-588.0,-178.0,49.0,83.0,330.0
SEQ03,74.0,4.0,11.0,59.0,15.0,52.0,41.0,55.0,81.0,96.0,...,-118.0,30.0,1005.0,44.0,1092.0,2861.0,1056.0,67.0,-54.0,271.0
SEQ04,43.0,16.0,58.0,78.0,37.0,50.0,16.0,55.0,76.0,98.0,...,9.0,97.0,1009.0,57.0,833.0,436.0,51.0,239.0,-56.0,859.0
SEQ05,91.0,4.0,40.0,138.0,30.0,40.0,19.0,26.0,45.0,128.0,...,-116.0,54.0,1012.0,51.0,537.0,-1543.0,-328.0,153.0,-78.0,470.0
SEQ06,51.0,23.0,62.0,97.0,43.0,74.0,27.0,43.0,39.0,101.0,...,-34.0,29.0,1003.0,70.0,479.0,-1761.0,-685.0,65.0,-31.0,257.0
SEQ07,110.0,41.0,59.0,13.0,43.0,82.0,20.0,41.0,77.0,66.0,...,-50.0,42.0,996.0,95.0,896.0,1110.0,284.0,68.0,-5.0,391.0
SEQ08,109.0,26.0,31.0,28.0,65.0,67.0,21.0,72.0,41.0,62.0,...,-25.0,42.0,993.0,103.0,935.0,1228.0,317.0,59.0,8.0,387.0
SEQ09,50.0,8.0,34.0,71.0,34.0,55.0,4.0,21.0,55.0,88.0,...,-89.0,27.0,1001.0,134.0,663.0,-77.0,-32.0,50.0,-49.0,238.0
SEQ10,37.0,15.0,39.0,20.0,42.0,75.0,10.0,63.0,48.0,178.0,...,-21.0,64.0,998.0,63.0,965.0,1822.0,309.0,89.0,8.0,589.0
